In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.preprocessing import LabelEncoder
from itertools import product


import time

In [39]:
test = pd.read_csv('../data/test.csv', index_col=0)
submit = pd.read_csv('../data/sample_submission.csv')
data = pd.read_pickle('data.pkl')


In [40]:
data.columns

Index(['month_idx', 'shop_id', 'item_id', 'item_cnt_month', 'cat_id',
       'type_code', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2',
       'item_cnt_month_lag_3', 'item_cnt_month_lag_6', 'item_cnt_month_lag_12',
       'date_avg_item_cnt_lag_1_x', 'date_item_avg_item_cnt_x',
       'date_avg_item_cnt_lag_1_y', 'date_item_avg_item_cnt_y',
       'date_avg_item_cnt_lag_1', 'date_shop_avg_item_cnt_lag_1',
       'date_shop_avg_item_cnt_lag_2', 'date_shop_avg_item_cnt_lag_4',
       'date_shop_avg_item_cnt_lag_6', 'date_shop_avg_item_cnt_lag_12',
       'date_cat_avg_item_cnt_lag_1', 'date_shop_cat_avg_item_cnt_lag_1',
       'delta_price_lag', 'delta_revenue_lag_1', 'month', 'days',
       'item_last_sale', 'item_shop_first_sale', 'item_first_sale'],
      dtype='object')

In [41]:
data = data[[
    'month_idx',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'cat_id',
    'type_code',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1_x', 
    'date_item_avg_item_cnt_x',
    'date_avg_item_cnt_lag_1_y',
    'date_item_avg_item_cnt_y',
    'date_avg_item_cnt_lag_1', 
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_4',
    'date_shop_avg_item_cnt_lag_6', 
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1', 
    'date_shop_cat_avg_item_cnt_lag_1', 
    'delta_price_lag',
    'delta_revenue_lag_1',
    'month',
    'days',
    'item_last_sale',
    'item_shop_first_sale',
    'item_first_sale'
]]


In [42]:
data.month_idx.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23], dtype=int8)

In [43]:
X_train = data[data.month_idx < 21].drop(['item_cnt_month'], axis=1)
y_train = data[data.month_idx < 21]['item_cnt_month']
X_valid = data[data.month_idx == 21].drop(['item_cnt_month'], axis=1)
y_valid  = data[data.month_idx == 21]['item_cnt_month']
X_test = data[data.month_idx ==23].drop(['item_cnt_month'], axis=1)


In [44]:
del data

In [45]:
import gc


In [46]:
gc.collect()

198

In [47]:

ts = time.time()

model = XGBRegressor(
    max_depth=8,
    n_estimators = 1000,
    min_child_weight = 300,
    colsample_bytree=0.8,
    subsample=0.8,
    eta=0.3,
    seed=42
)

model.fit(X_train, y_train,eval_metric="rmse", eval_set=[(X_train, y_train),(X_valid, y_valid)],
    verbose=True,
    early_stopping_rounds=10)

time.time() - ts


[0]	validation_0-rmse:1.57350	validation_1-rmse:1.70887
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:1.31451	validation_1-rmse:1.47890
[2]	validation_0-rmse:1.16856	validation_1-rmse:1.34630
[3]	validation_0-rmse:1.07721	validation_1-rmse:1.26870
[4]	validation_0-rmse:1.02488	validation_1-rmse:1.22708
[5]	validation_0-rmse:0.99714	validation_1-rmse:1.20124
[6]	validation_0-rmse:0.98178	validation_1-rmse:1.18853
[7]	validation_0-rmse:0.96603	validation_1-rmse:1.17804
[8]	validation_0-rmse:0.96039	validation_1-rmse:1.17291
[9]	validation_0-rmse:0.95477	validation_1-rmse:1.16809
[10]	validation_0-rmse:0.95172	validation_1-rmse:1.16565
[11]	validation_0-rmse:0.93995	validation_1-rmse:1.15603
[12]	validation_0-rmse:0.93441	validation_1-rmse:1.15350
[13]	validation_0-rmse:0.93210	validation_1-rmse:1.15274
[14]	validation_0-rmse:0.92986	validation_1-rmse:1.15

41.1927855014801

In [48]:
y_pred = model.predict(X_valid).clip(0,20)
y_test = model.predict(X_test).clip(0,20)

submission = pd.DataFrame({
    "ID": test.index,
    "item_cnt_month": y_test
})

submission.to_csv('xgb_submission.csv', index=False )


In [25]:
submit

,0,100
0,1,100
1,2,100
2,3,100
3,4,100
4,5,100
...,...,...
3054,3055,100
3055,3056,100
3056,3057,100
3057,3058,100


In [13]:
submission

,ID,item_cnt_month
0,0,0.360681
1,1,0.000000
2,2,0.383577
3,3,1.242619
4,4,2.760417
...,...,...
3055,3055,0.765410
3056,3056,0.000894
3057,3057,0.000894
3058,3058,0.000894
